In [2]:
!pip install langchain
!pip install langchain-community
!pip install langchain-text-splitters
!pip install transformers
!pip install sentence-transformers
!pip install faiss-cpu
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.6/330.6 kB 10.7 MB/s eta 0:00:00


In [1]:
import os

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [17]:
#Load PDF
loader = PyPDFLoader("A Novel Deep Learning Approach for Myocardial Infarction.pdf")
documents = loader.load()
print("Loaded pages:", len(documents))

#Split document into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
docs = text_splitter.split_documents(documents)
print("Total chunks created:", len(docs))

#Create embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

#Store embeddings in FAISS
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})        #will fetch top 4 relevant chunks during question answering.

Loaded pages: 19
Total chunks created: 231


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [18]:
#Load LLM FLAN-T5
model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

#Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


In [106]:
#RAG
question = "What are DL Classifiers used in this research?"

#Retrieve relevant chunks
relevant_docs = retriever.invoke(question)

#Combine top 2 chunks
context = "\n\n".join([doc.page_content for doc in relevant_docs[:2]])

#Trim long context
context = context[:1200]

In [107]:
#Build grounded prompt
prompt = f"""
You are an AI research assistant.
Answer the question clearly using the context below.

Context:
{context}

Question: {question}

Answer in 2-3 complete sentences:
"""

In [108]:
#GENERATION BLOCK
#Tokenize prompt
inputs = tokenizer(
    prompt,
    return_tensors="pt",
    truncation=True,
    max_length=512
)

#Move inputs to same device as model
inputs = {k: v.to(device) for k, v in inputs.items()}

#Generate output
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=False
)

answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

import textwrap
formatted_answer = textwrap.fill(answer, width=80)  #wrap at 80 chars
print("Answer:\n", formatted_answer)

Answer:
 (RNN, DNN, CNN, and LSTM)
